In [221]:
using JuMP, Cbc, Ipopt, Gurobi
o = Model(with_optimizer(Cbc.Optimizer))

┌ Warning: `with_optimizer` is deprecated. Adapt the following example to update your code:
│ `with_optimizer(Ipopt.Optimizer)` becomes `Ipopt.Optimizer`.
│   caller = top-level scope at In[221]:2
└ @ Core In[221]:2


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)

In [222]:
#viewers in millions in hours 9:21
v = [281.7
422
423
376.8
377.8
518.1
528
529
549.6
858.5
859.5
860.5
861.5]

13-element Array{Float64,1}:
 281.7
 422.0
 423.0
 376.8
 377.8
 518.1
 528.0
 529.0
 549.6
 858.5
 859.5
 860.5
 861.5

In [165]:
# qualifications variables
@variable(o, m_qual1[1:2, 9:21], Bin)      #men's qualification subdivision 1
@variable(o, m_qual2[1:2, 9:21], Bin)      #men's qualification subdivision 2
@variable(o, m_qual3[1:2, 9:21], Bin)      #men's qualification subdivision 3
@variable(o, w_qual1[1:2, 9:21], Bin)      #women's qualification subdivision 1
@variable(o, w_qual2[1:2, 9:21], Bin)      #women's qualification subdivision 2
@variable(o, w_qual3[1:2, 9:21], Bin)      #women's qualification subdivision 3

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, Base.OneTo(2)
    Dimension 2, 9:21
And data, a 2×13 Array{VariableRef,2}:
 w_qual3[1,9]  w_qual3[1,10]  w_qual3[1,11]  …  w_qual3[1,20]  w_qual3[1,21]
 w_qual3[2,9]  w_qual3[2,10]  w_qual3[2,11]     w_qual3[2,20]  w_qual3[2,21]

In [166]:
# QUALIFIERS

#the events cannot take place at the same time
for day in 1:2
    for hour in 9:21
        @constraint(o, sum(m_qual1[day, hour] + m_qual2[day, hour] + m_qual3[day, hour]) <= 1)
        @constraint(o, sum(w_qual1[day, hour] + w_qual2[day, hour] + w_qual3[day, hour]) <= 1)
    end
end

#make sure every event takes 2hrs, each gender is on a different day 
@variable(o, g, Bin)
men_qual = [m_qual1, m_qual2, m_qual3]
women_qual= [w_qual1, w_qual2, w_qual3]
for event in men_qual
    @constraint(o, sum(event[1,:]) == 2-2*g)
    @constraint(o, sum(event[2,:]) == 2*g)
end
for event in women_qual
    @constraint(o, sum(event[1,:]) == 2*g)
    @constraint(o, sum(event[2,:]) == 2-2*g)
end




`
#hours are consecutive now for m_qual1
@variable(o, t_m1_1[9:20], Bin) #binary variable denoting the start time of m_qual1 on day 1
@variable(o, t_m1_2[9:20], Bin) #binary variable denoting the start time of m_qual1 on day 2
@constraint(o, sum(t_m1_1[:]) == w)
@constraint(o, sum(t_m1_2[:]) == 1-w)
for hour in 9:20
    @constraint(o, sum(m_qual1[1, hour]+ m_qual1[1, hour+1]) >= 2*t_m1_1[hour])
    @constraint(o, sum(m_qual1[2, hour]+ m_qual1[2, hour+1]) == 2*t_m1_2[hour])
end

#hours are consecutive now for m_qual2
@variable(o, t_m2_1[9:20], Bin) #binary variable denoting the start time of m_qual2 on day 1
@variable(o, t_m2_2[9:20], Bin) #binary variable denoting the start time of m_qual2 on day 2
@constraint(o, sum(t_m2_1[:]) == w)
@constraint(o, sum(t_m2_2[:]) == 1-w)
for hour in 9:20
    @constraint(o, sum(m_qual2[1, hour]+ m_qual2[1, hour+1]) == 2*t_m2_1[hour])
    @constraint(o, sum(m_qual2[2, hour]+ m_qual2[2, hour+1]) == 2*t_m2_2[hour])
end

#hours are consecutive now for m_qual3
@variable(o, t_m3_1[9:20], Bin) #binary variable denoting the start time of m_qual3 on day 1
@variable(o, t_m3_2[9:20], Bin) #binary variable denoting the start time of m_qual3 on day 2
@constraint(o, sum(t_m3_1[:]) == w)
@constraint(o, sum(t_m3_2[:]) == 1-w)
for hour in 9:20
    @constraint(o, sum(m_qual3[1, hour]+ m_qual3[1, hour+1]) == 2*t_m3_1[hour])
    @constraint(o, sum(m_qual3[2, hour]+ m_qual3[2, hour+1]) == 2*t_m3_2[hour])
end

#hours are consecutive now for w_qual1
@variable(o, t_w1_1[9:20], Bin) #binary variable denoting the start time of w_qual1 on day 1
@variable(o, t_w1_2[9:20], Bin) #binary variable denoting the start time of w_qual1 on day 2
@constraint(o, sum(t_w1_1[:]) == 1-w)
@constraint(o, sum(t_w1_2[:]) == w)
for hour in 9:20
    @constraint(o, sum(w_qual1[1, hour]+ w_qual1[1, hour+1]) == 2*t_w1_1[hour])
    @constraint(o, sum(w_qual1[2, hour]+ w_qual1[2, hour+1]) == 2*t_w1_2[hour])
end

#hours are consecutive now for w_qual2
@variable(o, t_w2_1[9:20], Bin) #binary variable denoting the start time of w_qual2 on day 1
@variable(o, t_w2_2[9:20], Bin) #binary variable denoting the start time of w_qual2 on day 2
@constraint(o, sum(t_w2_1[:]) == 1-w)
@constraint(o, sum(t_w2_2[:]) == w)
for hour in 9:20
    @constraint(o, sum(w_qual2[1, hour]+ w_qual2[1, hour+1]) == 2*t_w2_1[hour])
    @constraint(o, sum(w_qual2[2, hour]+ w_qual2[2, hour+1]) == 2*t_w2_2[hour])
end
`
`
#hours are consecutive now for w_qual3
@variable(o, t_w3_1[9:20], Bin) #binary variable denoting the start time of w_qual3 on day 1
@variable(o, t_w3_2[9:20], Bin) #binary variable denoting the start time of w_qual3 on day 2
@constraint(o, sum(t_w3_1[:]) == g)
@constraint(o, sum(t_w3_2[:]) == 1-g)

for hour in 9:19
    if w_qual3[2, hour+1] == 1 && w_qual3[2, hour] != 1
        @constraint(o, sum(w_qual3[2, hour+2] + w_qual3[2, hour+1]) == 2)
    end
end 
`

In [167]:
#test the code above

objec = v'*(m_qual1[1,:])+v'*(m_qual1[2,:])+ v'*(m_qual2[1,:])+v'*(m_qual2[2,:])+ v'*(m_qual3[1,:])+v'*(m_qual3[2,:])
#womens qual
objec = objec + v'*(w_qual1[1,:])+v'*(w_qual1[2,:])+ v'*(w_qual2[1,:])+v'*(w_qual2[2,:])+ v'*(w_qual3[1,:])+v'*(w_qual3[2,:])

@objective(o, Max, objec)

281.7 m_qual1[1,9] + 422 m_qual1[1,10] + 423 m_qual1[1,11] + 376.8 m_qual1[1,12] + 377.8 m_qual1[1,13] + 518.1 m_qual1[1,14] + 528 m_qual1[1,15] + 529 m_qual1[1,16] + 549.6 m_qual1[1,17] + 858.5 m_qual1[1,18] + 859.5 m_qual1[1,19] + 860.5 m_qual1[1,20] + 861.5 m_qual1[1,21] + 281.7 m_qual1[2,9] + 422 m_qual1[2,10] + 423 m_qual1[2,11] + 376.8 m_qual1[2,12] + 377.8 m_qual1[2,13] + 518.1 m_qual1[2,14] + 528 m_qual1[2,15] + 529 m_qual1[2,16] + 549.6 m_qual1[2,17] + 858.5 m_qual1[2,18] + 859.5 m_qual1[2,19] + 860.5 m_qual1[2,20] + 861.5 m_qual1[2,21] + 281.7 m_qual2[1,9] + 422 m_qual2[1,10] + 423 m_qual2[1,11] + 376.8 m_qual2[1,12] + 377.8 m_qual2[1,13] + 518.1 m_qual2[1,14] + 528 m_qual2[1,15] + 529 m_qual2[1,16] + 549.6 m_qual2[1,17] + 858.5 m_qual2[1,18] + 859.5 m_qual2[1,19] + 860.5 m_qual2[1,20] + 861.5 m_qual2[1,21] + 281.7 m_qual2[2,9] + 422 m_qual2[2,10] + 423 m_qual2[2,11] + 376.8 m_qual2[2,12] + 377.8 m_qual2[2,13] + 518.1 m_qual2[2,14] + 528 m_qual2[2,15] + 529 m_qual2[2,16] + 54

In [168]:
optimize!(o)

for i in 9:21
    println(JuMP.value(w_qual3[2, i]))
end

0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
1.0
0.0
0.0
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 10326 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 52 strengthened rows, 0 substitutions
Cgl0004I processed model has 64 rows, 157 columns (157 integer (157 of which binary)) and 376 elements
Cbc0012I Integer solution of -9037.2 found by DiveCoefficient after 0 iterations and 0 nodes (0.06 seconds)
Cbc0001I Search completed - best objective -9037.200000000001, took 0 iterations and 0 nodes (0.06 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from -9037.2 to -9037.2
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 

In [171]:
# TEAMS FINALS
@variable(o, m_tf[4:5, 9:21], Bin)
@variable(o, w_tf[4:5, 9:21], Bin)

#team finals constraints
# men and women on different days
@variable(o, x, Bin)
@constraint(o, sum(m_tf[4, :]) == 2*x)
@constraint(o, sum(m_tf[5, :]) == 2-(2*x))
@constraint(o, sum(w_tf[4, :]) == 2-(2*x))
@constraint(o, sum(w_tf[5, :]) == 2*x)


`
# 2 hr for men
@variable(o, t_mt_4[9:20], Bin)     #binary variable denoting the start time of men's team finals on day 4
@variable(o, t_mt_5[9:20], Bin)     #binary variable denoting the start time of men's team finals on day 5
@constraint(o, sum(t_mt_4[:]) == x) 
@constraint(o, sum(t_mt_5[:]) == 1-x)
#makes sure the hours are consecutive
for hour in 9:20
    @constraint(o, sum(m_tf[4, hour]+ m_tf[4, hour+1]) == 2*t_mt_4[hour])
    @constraint(o, sum(m_tf[5, hour]+ m_tf[5, hour+1]) == 2*t_mt_5[hour])
end

#2hrs for women
@variable(o, t_wt_4[9:20], Bin)     #binary variable denoting the start time of women's team finals on day 4
@variable(o, t_wt_5[9:20], Bin)     #binary variable denoting the start time of women's team finals on day 5
@constraint(o, sum(t_wt_4[:]) == 1-x) 
@constraint(o, sum(t_wt_5[:]) == x)
#makes sure the hours are consecutive
for hour in 9:20
    @constraint(o, sum(w_tf[4, hour]+ w_tf[4, hour+1]) == 2*t_wt_4[hour])
    @constraint(o, sum(w_tf[5, hour]+ w_tf[5, hour+1]) == 2*t_wt_5[hour])
end
`

`'#' 2 hr for men '@variable(o,' 't_mt_4[9:20],' 'Bin)' '#binary' variable denoting the start time of 'mens team finals on day 4
@variable(o, t_mt_5[9:20], Bin)     #binary variable denoting the start time of mens' team finals on day 5 '@constraint(o,' 'sum(t_mt_4[:])' == 'x)' '@constraint(o,' 'sum(t_mt_5[:])' == '1-x)' '#makes' sure the hours are consecutive for hour in 9:20 '@constraint(o,' 'sum(m_tf[4,' 'hour]+' 'm_tf[4,' 'hour+1])' == '2*t_mt_4[hour])' '@constraint(o,' 'sum(m_tf[5,' 'hour]+' 'm_tf[5,' 'hour+1])' == '2*t_mt_5[hour])' end '#2hrs' for women '@variable(o,' 't_wt_4[9:20],' 'Bin)' '#binary' variable denoting the start time of 'womens team finals on day 4
@variable(o, t_wt_5[9:20], Bin)     #binary variable denoting the start time of womens' team finals on day 5 '@constraint(o,' 'sum(t_wt_4[:])' == '1-x)' '@constraint(o,' 'sum(t_wt_5[:])' == 'x)' '#makes' sure the hours are consecutive for hour in 9:20 '@constraint(o,' 'sum(w_tf[4,' 'hour]+' 'w_tf[4,' 'hour+1])' == '2*t_w

In [172]:
#only optimize teams finals(for testing purposes)
@objective(o, Max ,v'*m_tf[4,:]+v'*m_tf[5,:]+v'*w_tf[4,:]+v'*w_tf[5,:])

281.7 m_tf[4,9] + 422 m_tf[4,10] + 423 m_tf[4,11] + 376.8 m_tf[4,12] + 377.8 m_tf[4,13] + 518.1 m_tf[4,14] + 528 m_tf[4,15] + 529 m_tf[4,16] + 549.6 m_tf[4,17] + 858.5 m_tf[4,18] + 859.5 m_tf[4,19] + 860.5 m_tf[4,20] + 861.5 m_tf[4,21] + 281.7 m_tf[5,9] + 422 m_tf[5,10] + 423 m_tf[5,11] + 376.8 m_tf[5,12] + 377.8 m_tf[5,13] + 518.1 m_tf[5,14] + 528 m_tf[5,15] + 529 m_tf[5,16] + 549.6 m_tf[5,17] + 858.5 m_tf[5,18] + 859.5 m_tf[5,19] + 860.5 m_tf[5,20] + 861.5 m_tf[5,21] + 281.7 w_tf[4,9] + 422 w_tf[4,10] + 423 w_tf[4,11] + 376.8 w_tf[4,12] + 377.8 w_tf[4,13] + 518.1 w_tf[4,14] + 528 w_tf[4,15] + 529 w_tf[4,16] + 549.6 w_tf[4,17] + 858.5 w_tf[4,18] + 859.5 w_tf[4,19] + 860.5 w_tf[4,20] + 861.5 w_tf[4,21] + 281.7 w_tf[5,9] + 422 w_tf[5,10] + 423 w_tf[5,11] + 376.8 w_tf[5,12] + 377.8 w_tf[5,13] + 518.1 w_tf[5,14] + 528 w_tf[5,15] + 529 w_tf[5,16] + 549.6 w_tf[5,17] + 858.5 w_tf[5,18] + 859.5 w_tf[5,19] + 860.5 w_tf[5,20] + 861.5 w_tf[5,21]

In [180]:
optimize!(o)

for i in 9:21
    println(JuMP.value(w_tf[5, i]))
end

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 3444 - 0.00 seconds
Cbc0006I The LP relaxation is infeasible or too expensive
Cbc0045I Solution of -3444 already found by heuristic
Cuts at root node changed objective from 1.79769e+308 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
MixedIntegerRounding2 was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Fl

In [198]:
# INDIVIDUAL ALL AROUND
@variable(o, m_ar[6:7, 9:21], Bin)
@variable(o, w_ar[6:7, 9:21], Bin)

# men and women on different days
@variable(o, d, Bin)
@constraint(o, sum(m_ar[6, :]) == 3*d)
@constraint(o, sum(m_ar[7, :]) == 3-(3*d))
@constraint(o, sum(w_ar[6, :]) == 2-(2*d))
@constraint(o, sum(w_ar[7, :]) == 2*d)

`
# individual constraints
#2 hrs for women
@variable(o, t_war_6[9:20], Bin)   #binary variable denoting the start time of women's ind. all around on day 6
@variable(o, t_war_7[9:20], Bin)   #binary variable denoting the start time of women's ind. all around on day 7
@constraint(o, sum(t_war_6[:]) == 1-d)
@constraint(o, sum(t_war_7[:]) == d)
#make sure the hours are two in a row
for hour in 9:20
    @constraint(o, sum(w_ar[6, hour]+ w_ar[6, hour+ 1]) == 2*t_war_6[hour])
    @constraint(o, sum(w_ar[7, hour]+ w_ar[7, hour+ 1]) == 2*t_war_7[hour])
end
                
#3hrs for men
@variable(o, t_mar_6[9:20], Bin)   #binary variable denoting the start time of men's ind. all around on day 6
@variable(o, t_mar_7[9:20], Bin)   #binary variable denoting the start time of men's ind. all around on day 7
@constraint(o, sum(t_mar_6[:]) == d)
@constraint(o, sum(t_mar_7[:]) == 1-d)
#make sure the 3 hours are in a row
for hour in 9:19
    @constraint(o, sum(m_ar[6, hour]+ m_ar[6, hour+ 1] + m_ar[6, hour+2]) == 3*t_mar_6[hour])
    @constraint(o, sum(m_ar[7, hour]+ m_ar[7, hour+ 1] + m_ar[7, hour+2]) == 3*t_mar_7[hour])
end
`

`'#' individual constraints '#2' hrs for women '@variable(o,' 't_war_6[9:20],' 'Bin)' '#binary' variable denoting the start time of 'womens ind. all around on day 6
@variable(o, t_war_7[9:20], Bin)   #binary variable denoting the start time of womens' ind. all around on day 7 '@constraint(o,' 'sum(t_war_6[:])' == '1-d)' '@constraint(o,' 'sum(t_war_7[:])' == 'd)' '#make' sure the hours are two in a row for hour in 9:20 '@constraint(o,' 'sum(w_ar[6,' 'hour]+' 'w_ar[6,' hour+ '1])' == '2*t_war_6[hour])' '@constraint(o,' 'sum(w_ar[7,' 'hour]+' 'w_ar[7,' hour+ '1])' == '2*t_war_7[hour])' end '#3hrs' for men '@variable(o,' 't_mar_6[9:20],' 'Bin)' '#binary' variable denoting the start time of 'mens ind. all around on day 6
@variable(o, t_mar_7[9:20], Bin)   #binary variable denoting the start time of mens' ind. all around on day 7 '@constraint(o,' 'sum(t_mar_6[:])' == 'd)' '@constraint(o,' 'sum(t_mar_7[:])' == '1-d)' '#make' sure the 3 hours are in a row for hour in 9:19 '@constraint(o,' 'sum

In [199]:
objec = v'*m_ar[6,:]+v'*m_ar[7,:]+v'*w_ar[6,:]+v'*w_ar[7,:]
@objective(o, Max, objec)

281.7 m_ar[6,9] + 422 m_ar[6,10] + 423 m_ar[6,11] + 376.8 m_ar[6,12] + 377.8 m_ar[6,13] + 518.1 m_ar[6,14] + 528 m_ar[6,15] + 529 m_ar[6,16] + 549.6 m_ar[6,17] + 858.5 m_ar[6,18] + 859.5 m_ar[6,19] + 860.5 m_ar[6,20] + 861.5 m_ar[6,21] + 281.7 m_ar[7,9] + 422 m_ar[7,10] + 423 m_ar[7,11] + 376.8 m_ar[7,12] + 377.8 m_ar[7,13] + 518.1 m_ar[7,14] + 528 m_ar[7,15] + 529 m_ar[7,16] + 549.6 m_ar[7,17] + 858.5 m_ar[7,18] + 859.5 m_ar[7,19] + 860.5 m_ar[7,20] + 861.5 m_ar[7,21] + 281.7 w_ar[6,9] + 422 w_ar[6,10] + 423 w_ar[6,11] + 376.8 w_ar[6,12] + 377.8 w_ar[6,13] + 518.1 w_ar[6,14] + 528 w_ar[6,15] + 529 w_ar[6,16] + 549.6 w_ar[6,17] + 858.5 w_ar[6,18] + 859.5 w_ar[6,19] + 860.5 w_ar[6,20] + 861.5 w_ar[6,21] + 281.7 w_ar[7,9] + 422 w_ar[7,10] + 423 w_ar[7,11] + 376.8 w_ar[7,12] + 377.8 w_ar[7,13] + 518.1 w_ar[7,14] + 528 w_ar[7,15] + 529 w_ar[7,16] + 549.6 w_ar[7,17] + 858.5 w_ar[7,18] + 859.5 w_ar[7,19] + 860.5 w_ar[7,20] + 861.5 w_ar[7,21]

In [204]:
optimize!(o)

for i in 9:21
    println(JuMP.value(w_ar[7,i]))
end

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 4303.5 - 0.00 seconds
Cbc0006I The LP relaxation is infeasible or too expensive
Cbc0045I Solution of -4303.5 already found by heuristic
Cuts at root node changed objective from 1.79769e+308 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
MixedIntegerRounding2 was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds

In [223]:
# specific events
@variable(o, m_floor[9:11, 9:21], Bin)      # mens floor
@variable(o, m_vault[9:11, 9:21], Bin)      # mens vault
@variable(o, m_pb[9:11, 9:21], Bin)         # mens parallel bars
@variable(o, m_pommel[9:11, 9:21], Bin)     # mens pommel horse
@variable(o, m_rings[9:11, 9:21], Bin)      # mens rings
@variable(o, m_horizontal[9:11, 9:21], Bin) #mens horizontal bar
@variable(o, w_floor[9:11, 9:21], Bin)      # womens floor
@variable(o, w_vault[9:11, 9:21], Bin)      # womens vault
@variable(o, w_ub[9:11, 9:21], Bin)         # womens uneven bars
@variable(o, w_b[9:11, 9:21], Bin)          # womens beam


@constraint(o, sum(m_floor[9,i]+m_vault[9,i]+m_pb[9,i]+m_pommel[9,i]+m_rings[9,i]+m_horizontal[9,i]+ w_floor[9,i]+w_vault[9,i]+w_ub[9,i]+ w_b[9,i] for i in 9:21) >=3)
@constraint(o, sum(m_floor[9,i]+m_vault[9,i]+m_pb[9,i]+m_pommel[9,i]+m_rings[9,i]+m_horizontal[9,i]+ w_floor[9,i]+w_vault[9,i]+w_ub[9,i]+ w_b[9,i] for i in 9:21) <=4)

@constraint(o, sum(m_floor[10,i]+m_vault[10,i]+m_pb[10,i]+m_pommel[10,i]+m_rings[10,i]+m_horizontal[10,i]+ w_floor[10,i]+w_vault[10,i]+w_ub[10,i]+ w_b[10,i] for i in 9:21) >=3)
@constraint(o, sum(m_floor[10,i]+m_vault[10,i]+m_pb[10,i]+m_pommel[10,i]+m_rings[10,i]+m_horizontal[10,i]+ w_floor[10,i]+w_vault[10,i]+w_ub[10,i]+ w_b[10,i] for i in 9:21) <=4)

@constraint(o, sum(m_floor[11,i]+m_vault[11,i]+m_pb[11,i]+m_pommel[11,i]+m_rings[11,i]+m_horizontal[11,i]+ w_floor[11,i]+w_vault[11,i]+w_ub[11,i]+ w_b[11,i] for i in 9:21) >=3)
@constraint(o, sum(m_floor[11,i]+m_vault[11,i]+m_pb[11,i]+m_pommel[11,i]+m_rings[11,i]+m_horizontal[11,i]+ w_floor[11,i]+w_vault[11,i]+w_ub[11,i]+ w_b[11,i] for i in 9:21) <=4)


#at most two specific events can happen in an hour
#for each day of specific events, the sum of events in an hour cannot be more than 2
for i = 9:21
    @constraint(o, (m_floor[9,i]+m_vault[9,i]+m_pb[9,i]+m_pommel[9,i]+m_horizontal[9,i]+ w_floor[9,i]+w_vault[9,i]+w_ub[9,i]+ w_b[9,i]) <=2)
end

for i = 9:21
    @constraint(o, (m_floor[10,i]+m_vault[10,i]+m_pb[10,i]+m_pommel[10,i]+m_horizontal[10,i]+ w_floor[10,i]+w_vault[10,i]+w_ub[10,i]+ w_b[10,i]) <=2)
end

for i = 9:21
    @constraint(o, (m_floor[11,i]+m_vault[11,i]+m_pb[11,i]+m_pommel[11,i]+m_horizontal[11,i]+ w_floor[11,i]+w_vault[11,i]+w_ub[11,i]+ w_b[11,i]) <=2)
    end

#if womens floor is happening, mens floor cannot be happening (there is only one floor)
for i = 9:11
    for j = 9:21
        @constraint(o, (m_floor[i,j]+ w_floor[i,j] <=1))
    end
end

#women's and men's events cannot be at the same time
for day in 9:11
    for hour in 9:21
        @constraint(o, sum(w_floor[day, hour]+ w_vault[day, hour] + w_ub[day, hour] + w_b[day, hour]) <= 1)
        @constraint(o, sum(m_floor[day, hour]+ m_vault[day, hour] + m_pommel[day, hour] + m_horizontal[day, hour] + m_pb[day, hour] + m_rings[day, hour]) <= 1)
    end
end


`for day in 9:11 for hour in 9:21 '@constraint(o,' 'sum(w_floor[day,' 'hour]+' 'w_vault[day,' 'hour]' + 'w_ub[day,' 'hour]' + 'w_b[day,' 'hour])' '<=' '1)' '@constraint(o,' 'sum(m_floor[day,' 'hour]+' 'm_vault[day,' 'hour]' + 'm_pommel[day,' 'hour]' + 'm_horizontal[day,' 'hour]' + 'm_pb[day,' 'hour]' + 'm_rings[day,' 'hour])' '<=' '1)' end end`

In [224]:
`
#add up viewers at possible times of events to get objective to be maximized
#mens qual
objec = v'*(m_qual1[1,:])+v'*(m_qual1[2,:])+ v'*(m_qual2[1,:])+v'*(m_qual2[2,:])+ v'*(m_qual3[1,:])+v'*(m_qual3[2,:])
#womens qual
objec = objec + v'*(w_qual1[1,:])+v'*(w_qual1[2,:])+ v'*(w_qual2[1,:])+v'*(w_qual2[2,:])+ v'*(w_qual3[1,:])+v'*(w_qual3[2,:])
#team finals
objec = objec +v'*m_tf[4,:]+v'*m_tf[5,:]+v'*w_tf[4,:]+v'*w_tf[5,:]
#indiv all around
objec = objec +v'*m_ar[6,:]+v'*m_ar[7,:]+v'*w_ar[6,:]+v'*w_ar[7,:]
`
#specific events
#always a day of rest before finals (day 8)
objec = sum(v'*m_floor[i,:] for i in 9:11)
objec = objec +sum(v'*m_vault[i,:] for i in 9:11)
objec = objec +sum(v'*m_pb[i,:] for i in 9:11)
objec = objec +sum(v'*m_pommel[i,:] for i in 9:11)
objec = objec +sum(v'*m_horizontal[i,:] for i in 9:11)
objec = objec +sum(v'*m_rings[i,:] for i in 9:11)
objec = objec +sum(v'*w_floor[i,:] for i in 9:11)
objec = objec +sum(v'*w_vault[i,:] for i in 9:11)
objec = objec +sum(v'*w_ub[i,:] for i in 9:11)
objec = objec +sum(v'*w_b[i,:] for i in 9:11)


#objective

@objective(o, Max,  objec)


#take a look at our model
#o

optimize!(o)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 10335 - 0.00 seconds
Cgl0004I processed model has 3 rows, 76 columns (76 integer (37 of which binary)) and 76 elements
Cbc0012I Integer solution of -10335 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective -10335, took 0 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from -10335 to -10335
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was tried 0 times and created 0 cuts of which 0

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
